# Project : Web scrapping, knowledge base construction

## Part 1 : Web scrapping and knowledge base construction

In [50]:
from datasets import load_dataset
import nltk
import string
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import inflect

from nltk.stem.porter import PorterStemmer

from nltk.stem import WordNetLemmatizer

import sklearn_crfsuite
from sklearn_crfsuite import metrics

# nltk.download()

import sklearn_crfsuite
from sklearn_crfsuite import metrics
from datasets import load_dataset

import spacy
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support

import pandas as pd

import json

from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS

### Environment Setup

#### Datasets
We laod the CoNLL-2003 from Hugging Face

In [51]:
dataset = load_dataset("conll2003", trust_remote_code=True)

# Access the training, validation, and test sets
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

# Print the first example from the training set
print(train_dataset[0])

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


### Task 1 : Model for NER

#### 1. Text cleaning and prepocessing

In [52]:
def text_lowercase(text):
    return text.lower()

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

p = inflect.engine()
# convert number into words
def convert_number(text):
    # split string into list of words
    temp_str = text.split()
    # initialise empty list
    new_string = []

    for word in temp_str:
        # if word is a digit, convert the digit
        # to numbers and append into the new_string list
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)

        # append the word as it is
        else:
            new_string.append(word)

    # join the words of new_string to form a string
    temp_str = ' '.join(new_string)
    return temp_str

def replace_non_alphabetic_with_whitespace(text):
    modified_text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    return modified_text

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_whitespace(text):
    return  " ".join(text.split())

nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text


stemmer = PorterStemmer()

def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return stems


nltk.download('punkt')
nltk.download('wordnet') 
lemmatizer = WordNetLemmatizer()

def lemma_words(text):
    word_tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word) for word in word_tokens]
    return lemmas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\auria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\auria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
def preprocess_pipeline(example):
    text = " ".join(example['tokens'])

    text = text_lowercase(text)
    text = convert_number(text)
    text = replace_non_alphabetic_with_whitespace(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)

    # Convertir la liste en chaîne après avoir supprimé les stopwords
    text = " ".join(remove_stopwords(text))
    text = " ".join(stem_words(text))
    text = " ".join(lemma_words(text))
    
    processed_example = {'tokens': text.split(), 'ner_tags': example['ner_tags']}
    return processed_example

In [54]:
train_processed = train_dataset.map(preprocess_pipeline)
validation_processed = validation_dataset.map(preprocess_pipeline)
test_processed = test_dataset.map(preprocess_pipeline)

print(train_processed[:5])

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': ['0', '1', '2', '3', '4'], 'tokens': [['eu', 'reject', 'german', 'call', 'boycott', 'british', 'lamb'], ['peter', 'blackburn'], ['brussel'], ['european', 'commiss', 'said', 'thursday', 'disagre', 'german', 'advic', 'consum', 'shun', 'british', 'lamb', 'scientist', 'determin', 'whether', 'mad', 'cow', 'diseas', 'transmit', 'sheep'], ['germani', 'repres', 'european', 'union', 'veterinari', 'committe', 'werner', 'zwingmann', 'said', 'wednesday', 'consum', 'buy', 'sheepmeat', 'countri', 'britain', 'scientif', 'advic', 'clearer']], 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22], [22, 11], [12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 35, 24, 35, 37, 16, 21, 15, 24, 41, 15, 16, 21, 21, 20, 37, 40, 35, 21, 7], [22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 22, 38, 15, 22, 24, 20, 37, 21, 15, 24, 16, 15, 22, 15, 12, 16, 21, 38, 17, 7]], 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12], [11, 12], [11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 13, 11, 21, 22, 11, 12, 17

#### 2. Named entity recognition (NER)

##### Conditional Random Field (CRF) with sklearn_crfsuite

In [62]:
from itertools import chain

# Define features and labels for training
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.istitle()': word.istitle(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        if isinstance(word1, str):  # Vérifiez que word1 est une chaîne de caractères
            features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
            })
        else:
            print(f"Warning: word1 is not a string at index {i+1}: {word1}")
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

pos_tag_dict = dataset["train"].features["pos_tags"].feature.int2str
ner_tag_dict = dataset["train"].features["ner_tags"].feature.int2str

# Convert dataset into structured format with string POS and NER tags
train_sents = [
    list(zip(tokens, map(pos_tag_dict, pos_tags), map(ner_tag_dict, ner_tags)))
    for tokens, pos_tags, ner_tags in zip(train_dataset["tokens"], train_dataset["pos_tags"], train_dataset["ner_tags"])
]

test_sents = [
    list(zip(tokens, map(pos_tag_dict, pos_tags), map(ner_tag_dict, ner_tags)))
    for tokens, pos_tags, ner_tags in zip(test_dataset["tokens"], test_dataset["pos_tags"], test_dataset["ner_tags"])
]

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]


# Train the CRF model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)


# Evaluate the model
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

y_test_flat = list(chain(*y_test))
y_pred_flat = list(chain(*y_pred))

# Calculer les métriques
precision_crf, recall_crf, f1_score_crf, _ = precision_recall_fscore_support(
    y_test_flat, y_pred_flat, average='weighted'
)

print(f"Precision: {precision_crf:.4f}, Recall: {recall_crf:.4f}, F1-score: {f1_score_crf:.4f}")

              precision    recall  f1-score   support

       B-LOC       0.86      0.80      0.83      1668
      B-MISC       0.83      0.75      0.79       702
       B-ORG       0.77      0.73      0.75      1661
       B-PER       0.83      0.85      0.84      1617
       I-LOC       0.82      0.66      0.73       257
      I-MISC       0.71      0.68      0.69       216
       I-ORG       0.69      0.76      0.72       835
       I-PER       0.87      0.95      0.91      1156
           O       0.99      0.99      0.99     38323

    accuracy                           0.96     46435
   macro avg       0.82      0.80      0.80     46435
weighted avg       0.96      0.96      0.96     46435

Precision: 0.9557, Recall: 0.9559, F1-score: 0.9556


##### spaCy

In [63]:
true_entities = []
pred_entities = []
nlp = spacy.load("./best_ner_model")

for text, true_labels in zip(test_dataset["tokens"], test_dataset["ner_tags"]):
    text_str = " ".join(text)
    doc = nlp(text_str)
    
    # Convert true labels to named entity format
    true_labels = [ner_tag_dict(label) for label in true_labels]  
    true_entities.append(true_labels)
    
    # Initialiser les labels prédits avec 'O' (Outside)
    predicted_labels = ["O"] * len(text)
    
    # Associer les entités détectées aux tokens
    for ent in doc.ents:
        ent_text = ent.text.split()  # Séparer en tokens
        for token in ent_text:
            if token in text:  # Vérifier si le token est bien dans le texte d'origine
                idx = text.index(token)  # Trouver l'index du token
                predicted_labels[idx] = ent.label_  # Assigner l'entité
        
    pred_entities.append(predicted_labels)

precision_spaCy, recall_spaCy, f1_spaCy, _ = precision_recall_fscore_support(
    [label for sent in true_entities for label in sent], 
    [label for sent in pred_entities for label in sent], 
    average='weighted'
)

print(f"spaCy NER Model - Precision: {precision_spaCy:.4f}, Recall: {recall_spaCy:.4f}, F1-score: {f1_spaCy:.4f}")

c:\Users\auria\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.7.5 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


spaCy NER Model - Precision: 0.8090, Recall: 0.8168, F1-score: 0.8129


c:\Users\auria\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\auria\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##### Comparition of the performances

In [65]:
metrics_df = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1-Score'],
    'CRF': [precision_crf, recall_crf, f1_score_crf],
    'spaCy': [precision_spaCy, recall_spaCy, f1_spaCy]
})

metrics_df

,Metric,CRF,spaCy
0,Precision,0.955725,0.808966
1,Recall,0.955938,0.816776
2,F1-Score,0.955620,0.812852


We can see that the CRF model has a better score for these 3 metrics, which means that CRF performs better than spaCy overall for the named entity recognition task on our dataset.

##### Saving the extracted entities along their positions

In [10]:
import json

# Extraire les entités prédites par CRF
crf_entities = []
for sent, labels in zip(test_sents, y_pred):
    for i, label in enumerate(labels):
        if label != 'O':  # Exclure les tokens non annotés
            crf_entities.append({
                "text": sent[i][0],
                "entity": label,
                "position": i
            })

# Sauvegarde en JSON
with open("crf_entities.json", "w") as f:
    json.dump(crf_entities, f, indent=4)

print("Entities extracted from CRF model saved in crf_entities.json")

Entities extracted from CRF model saved in crf_entities.json


In [11]:
spacy_entities = []

for text in test_dataset["tokens"]:
    sentence = " ".join(text)
    doc = nlp(sentence)
    for ent in doc.ents:
        spacy_entities.append({
            "text": ent.text,
            "entity": ent.label_,
            "start": ent.start_char,
            "end": ent.end_char
        })

# Sauvegarde en JSON
with open("spacy_entities.json", "w") as f:
    json.dump(spacy_entities, f, indent=4)

print("Entities extracted from spaCy model saved in spacy_entities.json")

Entities extracted from spaCy model saved in spacy_entities.json


#### 3. Relation Extraction (RE)

In [ ]:
def extract_relations_from_dataset(dataset):
    nlp = spacy.load("en_core_web_sm")
    relations = []
    
    for example in dataset:
        text = " ".join(example["tokens"])  # Reconstituer la phrase à partir des tokens
        doc = nlp(text)
        
        for token in doc:
            if (token.dep_ == "nsubj" or token.dep_ == "nsubjpass") and token.head.dep_ == "ROOT":
                subject = token.text
                predicate = token.head.text  # Verbe principal                
                for child in token.head.children:
                    if child.dep_ in ["prep", "agent"]:  # Préposition ou agent
                        for obj in child.children:
                            if obj.dep_ == "pobj":  # Objet de la préposition
                                relations.append((subject, predicate, obj.text))
    
    return relations

# Exemple d'utilisation avec un dataset
from datasets import load_dataset

dataset = load_dataset("conll2003", trust_remote_code=True)
train_dataset = dataset['train']

extracted_relations = extract_relations_from_dataset(train_dataset)
print("Relations extraites:", extracted_relations[:10])

Relations extraites: [('Commission', 'said', 'Thursday'), ('He', 'said', 'Fischler'), ('Fischler', 'proposed', 'reports'), ('Palacio', 'accused', 'meeting'), ('farmers', 'denied', 'Thursday'), ('Jones', 'said', 'radio'), ('Germany', 'imported', 'Britain'), ('Hendrix', 'sells', '17,000'), ('draft', 'sold', '17,000'), ('draft', 'sold', 'Thursday')]


#### 4. Knowledge graph building

In [13]:
def knowledge_graph_from_relations(extracted_relations, word_to_find):
    # Create a new RDF graph
    g = Graph()

    # Define namespaces
    EX = Namespace("http://example.org/")

    # Ajouter les triplets au graphe
    for subj, pred, obj in extracted_relations:
        g.add((URIRef(EX[subj]), URIRef(EX[pred]), URIRef(EX[obj])))

    # Afficher les triplets RDF
    # print(g.serialize(format="turtle"))


    # Perform a SPARQL query
    query = """
    PREFIX ex: <http://example.org/>
    SELECT ?subject ?predicate
    WHERE {
        ?subject ?predicate <http://example.org/""" + word_to_find + """>
    }
    """
    for row in g.query(query):
        print(f"{row.subject} {row.predicate} ")

In [14]:
knowledge_graph_from_relations(extracted_relations, "Fischler")

http://example.org/He http://example.org/said 


In [71]:
text_star_wars = "Star Wars IV is a Movie where there are different kinds of creatures, like humans and wookies. Some creatures are Jedis; for instance, the human Luke is a Jedi, and Master Yoda - for whom the species is not known - is also a Jedi. The wookie named Chewbacca is Han's co-pilot on the Millennium Falcon starship. The speed of Millennium Falcon is 1.5 (above the speed of light!)"


In [78]:
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
displacy.render(nlp(text_star_wars), style="dep", jupyter=True, options={"compact": True, "distance": 100})

In [48]:
# Load spaCy's pre-trained NER model
nlp = spacy.load("./best_ner_model")

# Process the text with spaCy
doc = nlp(text_star_wars)

# Extract named entities
entities = [(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]
print("Extracted Entities:", entities)

c:\Users\auria\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.7.5 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Extracted Entities: [('Star Wars IV', 'ORG', 0, 12), ('Movie', 'PER', 18, 23), ('Jedis', 'PER', 114, 119), ('Luke', 'PER', 145, 149), ('Jedi', 'PER', 155, 159), ('Master Yoda', 'PER', 165, 176), ('Jedi', 'PER', 225, 229), ('Chewbacca', 'LOC', 248, 257), ('Millennium Falcon', 'ORG', 283, 300), ('Millennium Falcon', 'PER', 324, 341)]


In [68]:
def extract_relations_from_text(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    relations = []
    for token in doc:
        if token.dep_ in ["nsubj", "nsubjpass"] and token.head.pos_ in ["VERB", "AUX", "ROOT"]:
            subject = token.text
            predicate = token.head.text
            # Explorer les compléments pour identifier les objets et attributs
            for child in token.head.children:
                if child.dep_ in ["dobj", "attr", "acomp"]:
                    relations.append((subject, predicate, child.text))
                
                elif child.dep_ == "prep" or child.dep_ == "agent":  # Gérer les relations prépositionnelles
                    for obj in child.children:
                        if obj.dep_ == "pobj":
                            relations.append((subject, f"{predicate} {child.text}", obj.text))

        # Gérer les relations attributives directes (ex: "Luke is a Jedi")
        if token.dep_ in ["attr", "appos"] and token.head.dep_ in ["nsubj", "nsubjpass"]:
            relations.append((token.head.text, "is", token.text))
    
    return relations

extracted_relations = extract_relations_from_text(text_star_wars)
print("Relations extraites:", extracted_relations)

Relations extraites: [('IV', 'is', 'Movie'), ('creatures', 'are', 'Jedis'), ('Luke', 'is for', 'instance'), ('Luke', 'is', 'Jedi'), ('Yoda', 'is', 'Jedi'), ('species', 'known for', 'whom'), ('wookie', 'is', 'co'), ('wookie', 'is', '-'), ('wookie', 'is', 'pilot'), ('wookie', 'is on', 'starship'), ('speed', 'is', '1.5')]


### Task 2 : Pipeline for knowledge graph construction

#### 1. Fetch news articles

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os

NB_BOOKS = 10  # Number of books to scrape
EXPORT_PATH = "./books/"

url = "https://www.goodreads.com/list/show/1.Best_Books_Ever"  # Best Books Ever list

options = webdriver.FirefoxOptions()
options.headless = True  
browser = webdriver.Firefox(options=options)

try:
    browser.get(url)
    time.sleep(5)  # To allow time for page to load

    books_data = []
    while len(books_data) < NB_BOOKS:
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        # Find book listings
        for book_item in soup.select('a.bookTitle')[:NB_BOOKS - len(books_data)]:
            title = book_item.get_text(strip=True)
            link = "https://www.goodreads.com" + book_item['href']

            books_data.append((title, link))

        # If we haven't scraped enough books, go to the next page
        next_page = soup.select_one('.next a')
        if next_page:
            next_url = "https://www.goodreads.com" + next_page['href']
            browser.get(next_url)
            time.sleep(5)
        else:
            break

    
    if not os.path.exists(EXPORT_PATH):
        os.makedirs(EXPORT_PATH)

    for idx, (title, link) in enumerate(books_data[:NB_BOOKS]):
        browser.get(link)
        time.sleep(3) 

        book_soup = BeautifulSoup(browser.page_source, 'html.parser')

        author_tag = book_soup.select_one('.BookPageMetadataSection__contributor .ContributorLink__name')
        author = author_tag.get_text(strip=True) if author_tag else "Unknown"

        date_tag = book_soup.select_one('.BookDetails .FeaturedDetails p[data-testid="publicationInfo"]')
        release_date = date_tag.get_text(strip=True) if date_tag else "Unknown"

        summary_tag = book_soup.select_one('.BookPageMetadataSection__description')
        summary = summary_tag.get_text("\n", strip=True) if summary_tag else "No summary available."
        
        summary = summary.replace("Show more", "").strip()
        release_date=release_date.replace("First published", "").strip()

        # Save to file
        filename = f"{EXPORT_PATH}book_{idx+1}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"Title: {title}\n")
            f.write(f"Author: {author}\n")
            f.write(f"Release Date: {release_date}\n")
            f.write(f"URL: {link}\n\n")
            f.write(f"Summary:\n{summary}")

        print(f"Saved: {filename}")

finally:
    browser.quit()

print("Scraping complete!")

Saved: ./books/book_1.txt
Saved: ./books/book_2.txt
Saved: ./books/book_3.txt
Saved: ./books/book_4.txt
Saved: ./books/book_5.txt
Saved: ./books/book_6.txt
Saved: ./books/book_7.txt
Saved: ./books/book_8.txt
Saved: ./books/book_9.txt
Saved: ./books/book_10.txt
Scraping complete!


#### 2. Use methods from Task 1

## Part 2: Knowledge Graph Embedding

In [2]:
import numpy as np
import torch
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from SPARQLWrapper import SPARQLWrapper, JSON
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Knowledge Graph Embedding

#### 1. Setup PyKEEN

In [ ]:
# pip install pykeen torch

#### 2. Basic Embedding Pipeline

In [ ]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory

# Convert your RDF triples to PyKEEN format
triples = [(s, p, o) for s, p, o in g]
tf = TriplesFactory.from_labeled_triples(triples)

# Train a TransE model
results = pipeline(
    training=tf,
    model='TransE',
    epochs=100,
    learning_rate=0.01,
    training_batch_size=128,
    random_seed=42,
)

# Access embeddings
entity_embeddings = 
results.model.entity_embeddings.weight.detach().numpy()
relation_embeddings =
results.model.relation_embeddings.weight.detach().numpy()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\auria\\OneDrive - De Vinci\\Auriane\\Ecole\\ESILV\\A4\\S8\\DIA\\Web datamining and semantics\\Projet\\path_to_your_rdf_file.rdf'

#### 3. Evaluation

In [ ]:
# Evaluate entity similarities
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_similar_entities(entity_id, entity_embeddings, top_k=5):
    entity_vector = entity_embeddings[entity_id].reshape(1, -1)
    similarities = cosine_similarity(entity_vector, entity_embeddings)
    most_similar = np.argsort(similarities[0])[-top_k-1:-1][::-1]
    return most_similar

# Link prediction
results.evaluate()

### Tips for Small Knowledge Graphs

#### 1. Model Selection

- Use simpler models like TransE or DistMult for small datasets
- Avoid complex models that require large amounts of training data

#### 2. Training Configuration

- Increase the number of negative samples
- Use smaller embedding dimensions (e.g., 50-100 instead of 200+)
- Implement early stopping to prevent overfitting

#### 3. Example Configuration

In [ ]:
from pykeen.pipeline import pipeline

results = pipeline(
    training=tf,
    model='TransE',
    epochs=100,
    embedding_dim=50,
    learning_rate=0.01,
    training_batch_size=32,
    num_negs_per_pos=10,
    early_stopping=True,
    early_stopping_patience=5,
    random_seed=42,
)

#### 4. Quality Assessment

- Compare embeddings before and after data augmentation
- Use visualization tools to inspect entity clusters
- Validate predictions against domain knowledge

### Challenge : Limited Entity Count

#### 1. Data Augmentation Strategies

- Use external knowledge bases like DBpedia to add related entities and relations
- Implement entity linking to map your entities to Wikidata/DBpedia entities Here is an example code. You may need to improve it with entitiy linking to avoid ambiguity brought by the entity names. (see slide 82 in Lecture 5: Web mining and Information retrieval for knowledge graph building)

#### 2. Integration with External Knowledge

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
def enrich_with_dbpedia(entity_name):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = """
    SELECT DISTINCT ?related ?relation WHERE {
    ?s rdfs:label "%s"@en .
    ?s ?relation ?related .
    ?related rdfs:label ?label .
    FILTER(LANG(?label) = 'en')
    } LIMIT 10
    """ % entity_name

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

### Detailed Embedding Evaluation Guide

#### 1. Data Splitting

In [ ]:
from pykeen.triples import TriplesFactory

# Split your triples into training (80%), validation (10%), and test (10%) sets
training, validation, testing =
TriplesFactory.from_labeled_triples(triples).split([0.8, 0.1, 0.1])

print(f"Training triples: {training.num_triples}")
print(f"Validation triples: {validation.num_triples}")
print(f"Testing triples: {testing.num_triples}")

#### 2. Model Training with Multiple Architecture

In [ ]:
from pykeen.pipeline import pipeline

# Dictionary to store results
model_results = {}

# Test different models
models = ['TransE', 'DistMult', 'ComplEx']
for model_name in models:
    results = pipeline(
    training=training,
    validation=validation,
    testing=testing,
    model=model_name,
    epochs=100,
    embedding_dim=50,
    training_kwargs=dict(batch_size=32),
    random_seed=42,
    )
    model_results[model_name] = results

#### 3. Evaluation Metrics

In [ ]:
def evaluate_model(results, model_name):
    metrics = results.metric_results.to_dict()
    print(f"\nResults for {model_name}:")
    print(f"Mean Rank: {metrics['both']['mean_rank']:.2f}")
    print(f"Mean Reciprocal Rank: {metrics['both']['mean_reciprocal_rank']:.4f}")
    print(f"Hits@1: {metrics['both']['hits_at_1']:.4f}")
    print(f"Hits@3: {metrics['both']['hits_at_3']:.4f}")
    print(f"Hits@10: {metrics['both']['hits_at_10']:.4f}")
for model_name, results in model_results.items():
    evaluate_model(results, model_name)


#### 4. Entity Similarity Analysis

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def analyze_entity_neighborhood(model, entity_id, k=5):
    # Get entity embeddings
    entity_embeddings = model.entity_embeddings.weight.detach().numpy()
    entity_labels = model.triples_factory.entity_labels

    # Calculate similarities
    similarities = cosine_similarity([entity_embeddings[entity_id]], entity_embeddings)[0]
    most_similar = np.argsort(similarities)[-k-1:-1][::-1]

    print(f"\nMost similar entities to {entity_labels[entity_id]}:")
    for idx in most_similar:
        print(f"{entity_labels[idx]}: {similarities[idx]:.4f}")

#### 5. Link Prediction Examples

In [ ]:
import torch

def predict_tail_entities(model, head, relation, k=5):
    # Get predictions
    predictions = model.predict_scores(
        heads=torch.tensor([head]),
        relations=torch.tensor([relation]),
    )

    # Get top k predictions
    top_tails = torch.topk(predictions, k=k, dim=1)

    return [(model.triples_factory.entity_labels[idx.item()],score.item())
    for idx, score in zip(top_tails.indices[0], top_tails.values[0])]

#### 6. Visualization of Embeddings

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
def visualize_embeddings(model, entity_types=None):
    # Get embeddings and reduce dimensionality
    embeddings = model.entity_embeddings.weight.detach().numpy()
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)

    # Plot
    plt.figure(figsize=(10, 10))
    if entity_types is None:
        plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.5)
    else:
        # Color by entity type
        for entity_type, indices in entity_types.items():
            plt.scatter(reduced_embeddings[indices, 0],
                reduced_embeddings[indices, 1],
                label=entity_type, alpha=0.5)
    plt.legend()

    plt.title("Entity Embeddings Visualization")
    plt.show()

#### 7. Performance Comparison Template

In [ ]:
def create_comparison_table(model_results):
    metrics = ['mean_rank', 'mean_reciprocal_rank', 'hits_at_10']
    comparison = {model: {} for model in model_results.keys()}

    for model_name, results in model_results.items():
        for metric in metrics:
            comparison[model_name][metric] = results.metric_results.to_dict()['both'][metric]

    return pd.DataFrame(comparison).round(4)